In [15]:
import pandas as pd
import pickle
from pathlib import Path
import os
import sys

<h3>Necessary Files</h3>
<p>
   - EIA 923: Generation and Fuel Data Sheet<br>
   - EIA 923: Fuel Receipts and Costs<br>
   - EIA 860: 2__Plant_Y2017<br>
   - EIA 860: 3_1_Generator_Y2017 - Sheet: Operable<br>
 </p>

In [2]:
def file_check(list_o_files):
    need = []
    for i in list_o_files:
        exists = os.path.isfile(i)
        if exists:
            print('{0} exists'.format(i))
        else:
            print('{0} does not exist.'.format(i))
            need.append(i)
    return need

In [3]:
def open_and_pickle(file_per_design, needed_files, specific_Sheet, rows2skip, year):
    
    read = pd.read_excel(file_per_design,
                         sheet_name = specific_Sheet,
                         skiprows = rows2skip)
    read.to_pickle('../pickleJar/{0}/{1}.pkl'.format(year, specific_Sheet.replace(" ", "_")))

In [96]:
def EIA_DataScrub(year):
    
    '''Check for Directory and Create if needed'''
    exists = os.path.isdir('../pickleJar/{0}/'.format(year))
    if exists:
        None
    else:
        os.makedirs('../pickleJar/{0}/'.format(year))
    '''Files'''
    EIA923 = '../f923_{0}/EIA923_Schedules_2_3_4_5_M_12_{0}_Final_Revision.xlsx'.format(year)
    EIA860_Gen = '../EIA860/3_1_Generator_Y{0}.xlsx'.format(year)
    EIA860_Pla = '../EIA860/2___Plant_Y{0}.xlsx'.format(year)
    Files = [EIA923, EIA860_Gen, EIA860_Pla]
    '''Pickles'''
    EIA923_FRC = '../pickleJar/{0}/Page_5_Fuel_Receipts_and_Costs.pkl'.format(year)
    EIA923_GFD = '../pickleJar/{0}/Page_1_Generation_and_Fuel_Data.pkl'.format(year)
    EIA860_nt = '../pickleJar/{0}/Plant.pkl'.format(year)
    EIA860_erator = '../pickleJar/{0}/Operable.pkl'.format(year)
    Pickles = [EIA923_FRC, EIA923_GFD, EIA860_nt, EIA860_erator]
    '''Sheets'''
    FRC_Sheet = 'Page 5 Fuel Receipts and Costs'
    GFD_Sheet = 'Page 1 Generation and Fuel Data'
    erator_Sheet = 'Operable'
    sheets = [FRC_Sheet, GFD_Sheet, erator_Sheet]
    
    needed = file_check(Pickles)
    
    if not needed:
        print('All Files are Ready!')
    else:
        print('Missing files. Checking for source files...')
        sources = file_check(Files)
        if not sources:
            print('All source files exist! Pickling the needed files...')
            for i in needed:
                if 'Fuel_Receipts' in i:
                    open_and_pickle(EIA923, i, sheets[0], 4, year)
                elif 'Generation' in i:
                    open_and_pickle(EIA923, i, sheets[1], 5, year)
                elif 'Plant' in i:
                    open_and_pickle(EIA860_Pla, i, 'Plant', 1, year)
                elif 'Operable' in i:
                    open_and_pickle(EIA860_Gen, i, sheets[2], 1, year)
                else:
                    print('A Fatal Error Occured')
                    sys.exit()
        else:
            print('The following source files are needed:')
            for i in sources:
                print(i)
                print('files can be obtained from https://www.EIA.gov/')
                sys.exit()
    '''Open the Pickle Files'''
    # Fuel Receipts and Costs
    EIA923_FRC_Data = pd.read_pickle(EIA923_FRC)
    # Generation and Fuel Data
    EIA923_GFD_Data = pd.read_pickle(EIA923_GFD)
    # EIA 860 Plant level data
    EIA860_nt_Data = pd.read_pickle(EIA860_nt)
    # EIA 860 Operable plants
    EIA860_erator_Data = pd.read_pickle(EIA860_erator)
    
    '''Start Scrubbing!'''
    # scrub generation and fuel data
    EIA923_GFD_Data = (EIA923_GFD_Data[['Plant Id',
                                        'Plant Name',
                                        'Sector Name']].
                      groupby(['Plant Id',
                               'Plant Name',
                               'Sector Name'])['Sector Name'].
                      count().to_frame().drop(columns=['Sector Name']).
                      reset_index(level=['Plant Id',
                                         'Plant Name',
                                         'Sector Name']))
    # scrub Fuel Receipts and Costs
    EIA923_FRC_Data = (EIA923_FRC_Data.
                       loc[EIA923_FRC_Data['ENERGY_SOURCE'].isin(['NG'])])
    EIA923_FRC_Data = (EIA923_FRC_Data[['Plant Id',
                             'Plant Name',
                             'Plant State',
                             'FUEL_GROUP',
                             'QUANTITY']].
                       groupby(['Plant Id',
                                'Plant Name',
                                'Plant State',
                                'FUEL_GROUP'])['QUANTITY'].sum().
                       to_frame().reset_index(level=['Plant Id',
                                                     'Plant Name',
                                                     'Plant State',
                                                     'FUEL_GROUP',]))
    # Scrub EIA 860 Generator Data
    EIA860_erator_Data['Technology'] = (EIA860_erator_Data['Technology'].
                                        fillna('invalid').
                                        apply(lambda x: x if 'Natural Gas' in x else 'Not Important'))
    EIA860_erator_Data = (EIA860_erator_Data[['Plant Code',
                                              'Plant Name',
                                              'State',
                                              'Technology',
                                              'Prime Mover',
                                              'Nameplate Capacity (MW)']].
                          loc[EIA860_erator_Data['Technology'] != 'Not Important'].
                          groupby(['Plant Code', 'Plant Name'])['Nameplate Capacity (MW)'].
                          sum().to_frame().reset_index(level='Plant Name'))
    # Scrub EIA 860 Plant Level Data
    EIA860_nt_Data = EIA860_nt_Data[['Plant Code',
                                     'Latitude',
                                     'Longitude',
                                     'Natural Gas LDC Name',
                                     'Natural Gas Pipeline Name 1',
                                     'Natural Gas Pipeline Name 2',
                                     'Natural Gas Pipeline Name 3',
                                     'Pipeline Notes']]
    
    '''Merge Data into one Dataset'''
    # Merge FRC and GFD data
    EIA923_FRC_Data = (EIA923_FRC_Data.
    merge(EIA923_GFD_Data,
         how = 'left',
         left_on = 'Plant Id',
         right_on = 'Plant Id',
         suffixes = ('', '_y')).drop(columns=['Plant Name_y']))
    # Merge (FRC/GFD) and EIA860 Gen Data
    EIA923_FRC_Data = (EIA923_FRC_Data.
    merge(EIA860_erator_Data,
         how = 'left',
         left_on = 'Plant Id',
         right_on = 'Plant Code',
         suffixes = ('', '_y')).drop(columns=['Plant Name_y']))
    # Merge (FRC/GFD/EIA860 Gen) and EIA860 Plants
    complete = (EIA923_FRC_Data.
    merge(EIA860_nt_Data,
          how = 'left',
          left_on = 'Plant Id',
          right_on = 'Plant Code',
          suffixes = ('', '_y')).drop(columns=['Plant Code']))
    
    return complete